# Decomposoitions and Plots

In [ ]:
import numpy as np
import h5py
import torch
from tensorly.decomposition import non_negative_parafac_hals

In [2]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Fri Jan  9 12:42:01 2026       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   49C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [6]:
# Import google drive mounting module
from google.colab import drive

# Import os/path lib to navigate the colab directory.
import os
import pathlib


# Mount Google Drive at the default location
drive.mount('/content/drive', force_remount=False)
# Define the path to your desired folder
path = '/content/drive/My Drive/OPM-MEG'
# Change to that directory
os.chdir(path)
# Verify the current working directory
print("Current Directory:", os.getcwd())

Mounted at /content/drive
Current Directory: /content/drive/My Drive/OPM-MEG


In [7]:
!git clone https://github.com/hug0-w/Tensor-Decompositions-OPM-MEG/
!pip install tensorly

Cloning into 'Tensor-Decompositions-OPM-MEG'...
remote: Enumerating objects: 66, done.
remote: Counting objects: 100% (66/66), done.
remote: Compressing objects: 100% (45/45), done.
remote: Total 66 (delta 15), reused 64 (delta 13), pack-reused 0 (from 0)
Receiving objects: 100% (66/66), 83.76 KiB | 708.00 KiB/s, done.
Resolving deltas: 100% (15/15), done.


# Choice

In [ ]:
mat_path = "choice_pow_trial_chan_fbin_time_ds500_tpos.mat"

with h5py.File(mat_path, "r") as f:

    # --- main data ---
    # HDF5 reads MATLAB v7.3 arrays with transposed dimensions.
    # MATLAB Saved: [Trials, Chan, Freqs, Time]
    # Python Reads: (Time, Freqs, Chan, Trials)
    Pow = np.array(f["Pow"])

    freq = np.array(f["freq"]).squeeze()
    time = np.array(f["time"]).squeeze()

    # Removed: fbins = np.array(f["fbins"])
    freqs_hz = np.array(f["freqs_hz"]).squeeze()

    ds_fs = float(np.array(f["ds_fs"]).squeeze())
    fs_orig = float(np.array(f["fs_orig"]).squeeze())

    # --- channel labels (MATLAB cellstr) ---
    # MATLAB cell arrays of strings are stored as object references in HDF5
    ch_names_refs = f["chan_lbl"][()]
    chan_lbl = []
    for r in ch_names_refs.flatten():
        s = f[r][()]
        # MATLAB v7.3 stores strings as uint16 (utf-16le)
        chan_lbl.append(s.tobytes().decode("utf-16le").rstrip("\x00"))

    # --- MNE metadata ---
    mne_grp = f["mne"]

    ch_names_refs = mne_grp["ch_names"][()]
    mne_ch_names = []
    for r in ch_names_refs.flatten():
        s = f[r][()]
        mne_ch_names.append(s.tobytes().decode("utf-16le").rstrip("\x00"))

    ch_pos_m = np.array(mne_grp["ch_pos_m"], dtype=float)
    # MATLAB stored (N, 3), HDF5 reads (3, N). Transpose to get (N_chan, 3).
    if ch_pos_m.shape[0] == 3 and ch_pos_m.shape[1] != 3:
        ch_pos_m = ch_pos_m.T

    # Decode coordinate frame string (e.g., 'head')
    coord_frame_data = mne_grp["coord_frame"][()]
    try:
        coord_frame = coord_frame_data.tobytes().decode("utf-16le").rstrip("\x00")
    except AttributeError:
        # Fallback if it loaded as a simple byte string or char
        coord_frame = str(coord_frame_data)

In [ ]:
Pow_choice = Pow.T
Pos_2d_choice = ch_pos_m[[:, :2]]

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# 1. Apply Option 2: Shifted Log (dB-like)
# Add a tiny epsilon to avoiding log(0) issues if you have exact zeros
epsilon = 1e-20 
log_Pow_choice = np.log10(Pow_choice + epsilon)

# Shift so the minimum is 0 (optional, but helpful for non-negative decomposition)
# This makes it similar to a dB scale relative to the noise floor
log_Pow_shifted_choice = log_Pow_choice - np.min(log_Pow_choice)

# 2. Plot the Distribution
plt.figure(figsize=(10, 5))
plt.hist(log_Pow_choice.ravel(), bins=100, density=True, color='blue', alpha=0.7)
plt.hist(log_Pow_shifted_choice.ravel(), bins=100, density=True, color='red', alpha=0.7)

plt.title("Distribution of Log-Normalized Power (Shifted Log10)")
plt.xlabel("Log Power (Arbitrary Units)")
plt.ylabel("Density")
plt.grid(True, alpha=0.3)
plt.show()

# 3. Check for Non-Negativity (Required for Non-Negative CP)
print(f"Minimum Value: {np.min(log_Pow)}")
print(f"Is all positive? {np.all(log_Pow >= 0)}")

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
log_Pow_shifted_torch = torch.as_tensor(Pow.T, device=device, dtype=torch.float32)

In [ ]:
x = 15


cp_tensor_choice = non_negative_parafac_hals(
                log_Pow_shifted_choice,
                rank=x,
                init="random",
                n_iter_max=2000,
                tol=1e-6,
                random_state=5  # Ensure different random init
            )

In [ ]:
weights_choice, factors_choice = cp_tensor_choice
A_choice, B_choice, C_choice, D_choice = factors_choice

In [ ]:
from src.utils.plots import plot_factors

In [ ]:
plot_factors(A_choice,B_choice,C_choice,D_choice,Pos_2d_choice)

# Outcome